In [1]:
# =========================
# GPR – Minimal/Debug Tuning-Pipeline
# =========================
# - Sehr kleines Grid, schneller Lauf
# - GlobalConfig(preset="debug")
# - Gleiches API/Flow wie bei den anderen Modellen
# =========================

import os, sys
from pathlib import Path
from itertools import product
import numpy as np
import pandas as pd

# --- 1) Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- 2) Repo-Imports ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
from src.models.GPR import ForecastModel

# --- 3) Master-Schalter & Meta ---
USE_DYNAMIC_FI_PIPELINE = False
SEED = 42

MODEL_NAME = "gpr_dynamic_fi_debug" if USE_DYNAMIC_FI_PIPELINE else "gpr_debug"
outputs_for_model(MODEL_NAME)
print(f"Modell {MODEL_NAME} wird getunt.")

# --- 4) Daten ---
y    = load_target()
X_ifo = load_ifo_features()

FI_PATH = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
if USE_DYNAMIC_FI_PIPELINE:
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_PATH)
        rolling_imp   = load_rolling_importance(base_dir=FI_PATH)
        idx = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi = y.loc[idx]
        X_full_lagged = X_full_lagged.loc[idx]
        rolling_imp   = rolling_imp.loc[idx]
        print("Dynamic-FI Daten gefunden. Shapes:", X_full_lagged.shape, rolling_imp.shape, y_fi.shape)
    except FileNotFoundError as e:
        print("FEHLER beim Laden der Dynamic-FI Artefakte:", e)
        print("Bitte erst FI-Jobs laufen lassen.")
        raise
else:
    idx = y.index.intersection(X_ifo.index)
    y, X_ifo = y.loc[idx], X_ifo.loc[idx]
    X_full_lagged = rolling_imp = y_fi = None
    print("Full-FE Daten geladen. Shapes:", X_ifo.shape, y.shape)

# --- 5) Debug-Config ---
def base_cfg_debug() -> GlobalConfig:
    cfg = GlobalConfig(preset="debug")
    cfg.policy_window   = 6
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 2
    return cfg

cfg0 = base_cfg_debug()

# --- 6) Corr-Helper (nur expanding im Debug) ---
def make_corr_spec(kind: str) -> dict:
    if kind == "expanding":
        return dict(DEFAULT_CORR_SPEC)
    raise ValueError("Nur 'expanding' im Debug-Notebook vorgesehen.")

# --- 7) Mini-Grids ---
if USE_DYNAMIC_FI_PIPELINE:
    print("Erstelle Mini-Grid für 'Dynamic FI' (Debug)...")

    N_FEATURES_TO_USE = 10

    kernel_list            = ["rbf"]
    ard_list               = [True]
    length_scale_init_list = [1.0]
    nu_list                = [1.5]      # irrelevant für rbf, aber konsistent
    noise_alpha_list       = [1e-6]
    n_restarts_list        = [0]
    normalize_y_list       = [True]
    optimizer_list         = ["fmin_l_bfgs_b"]
    weighting_opts         = [{"sample_weight_decay": None}]

    def build_model_grid_dynamic_fi():
        grid = []
        for k, ard, ls0, nu, a, nr, norm, opt in product(
            kernel_list, ard_list, length_scale_init_list, nu_list,
            noise_alpha_list, n_restarts_list, normalize_y_list, optimizer_list
        ):
            base = {
                "n_features_to_use": N_FEATURES_TO_USE,
                "kernel": k, "ard": ard, "length_scale_init": ls0, "nu": nu,
                "noise_alpha": a, "n_restarts_optimizer": nr, "normalize_y": norm,
                "optimizer": opt, "seed": SEED,
            }
            for w in weighting_opts:
                hp = dict(base); hp.update(w); grid.append(hp)
        return grid

    model_grid = build_model_grid_dynamic_fi()

else:
    print("Erstelle Mini-Grid für 'Full FE' (Debug)...")

    corr_options = [("expanding", make_corr_spec("expanding"))]
    lag_candidates_list   = [(1, 2, 3)]
    top_k_lags_list       = [1]
    use_rm3_list          = [True]
    k1_topk_list          = [100]
    redundancy_param_list = [0.90]
    dr_options_list       = [{"dr_method": "none"}]

    kernel_list            = ["rbf"]
    ard_list               = [True]
    length_scale_init_list = [1.0]
    nu_list                = [1.5]
    noise_alpha_list       = [1e-6]
    n_restarts_list        = [0]
    normalize_y_list       = [True]
    optimizer_list         = ["fmin_l_bfgs_b"]

    target_block_options = [None]
    weighting_opts       = [{"sample_weight_decay": None}]

    def build_model_grid_full_fe():
        hp_grid = []
        fe_lists = [
            lag_candidates_list, top_k_lags_list, use_rm3_list,
            k1_topk_list, redundancy_param_list, dr_options_list
        ]
        for (corr_tag, corr_spec) in corr_options:
            for (lags, k_lags, rm3, k1, red, dr_opt) in product(*fe_lists):
                base_fe = {
                    "lag_candidates": lags,
                    "top_k_lags_per_feature": k_lags,
                    "use_rm3": rm3,
                    "k1_topk": k1,
                    "redundancy_param": red,
                    **dr_opt,
                    "corr_tag": corr_tag,
                    "corr_spec": corr_spec,
                }
                for k, ard, ls0, nu, a, nr, norm, opt in product(
                    kernel_list, ard_list, length_scale_init_list, nu_list,
                    noise_alpha_list, n_restarts_list, normalize_y_list, optimizer_list
                ):
                    base_model = {
                        "kernel": k, "ard": ard, "length_scale_init": ls0, "nu": nu,
                        "noise_alpha": a, "n_restarts_optimizer": nr,
                        "normalize_y": norm, "optimizer": opt, "seed": SEED,
                    }
                    for block_set in target_block_options:
                        for w in weighting_opts:
                            hp = {**base_fe, **base_model, "target_block_set": block_set, **w}
                            hp_grid.append(hp)
        return hp_grid

    model_grid = build_model_grid_full_fe()

print("HP-Kombinationen (Mini):", len(model_grid))
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid ist leer")

# --- 8) Stage A/B ---
if model_grid:
    if USE_DYNAMIC_FI_PIPELINE:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,  # Platzhalter
            y=y_fi,
            cfg=cfg0,
            keep_top_k_final=min(3, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//3),
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,  # Platzhalter
            y=y_fi,
            cfg=cfg0,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
    else:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,
            y=y,
            cfg=cfg0,
            keep_top_k_final=min(3, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//3),
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,
            y=y,
            cfg=cfg0,
        )
else:
    print("Keine gültigen HP-Kombinationen gefunden, Stages übersprungen.")

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Modell gpr_debug wird getunt.
INFO in load_ifo_features: Renaming columns to ensure validity.
Full-FE Daten geladen. Shapes: (407, 2160) (407,)
Erstelle Mini-Grid für 'Full FE' (Debug)...
HP-Kombinationen (Mini): 1
Erstes HP-Set: {'lag_candidates': (1, 2, 3), 'top_k_lags_per_feature': 1, 'use_rm3': True, 'k1_topk': 100, 'redundancy_param': 0.9, 'dr_method': 'none', 'corr_tag': 'expanding', 'corr_spec': {'mode': 'expanding', 'window': None, 'lam': None}, 'kernel': 'rbf', 'ard': True, 'length_scale_init': 1.0, 'nu': 1.5, 'noise_alpha': 1e-06, 'n_restarts_optimizer': 0, 'normalize_y': True, 'optimizer': 'fmin_l_bfgs_b', 'seed': 42, 'target_block_set': None, 'sample_weight_decay': None}
[Stage A][Block 1] train_end=48, OOS=49-60 | configs=1
  - Config 1/1
    -> Survivors after block 1: 1
[Stage A][Block 2] train_end=60, OOS=61-72 | configs=1
  - Config 1/1


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


    -> Survivors after block 2: 1
[Stage A] Shortlist saved with 1 configs.
[Stage B] t=73 (I_t=74) | incumbent=1/1
[Stage B] t=74 (I_t=75) | incumbent=1/1
[Stage B] t=75 (I_t=76) | incumbent=1/1
    active=1 | t=75: y=3.0184, ŷ=-0.0526 (WRMSEs=[1.9420015326346982], gain=0.000%)
[Stage B] t=76 (I_t=77) | incumbent=1/1
    active=1 | t=76: y=1.2739, ŷ=-0.0117 (WRMSEs=[1.7889326846439115], gain=0.000%)
[Stage B] t=77 (I_t=78) | incumbent=1/1
    active=1 | t=77: y=-3.2704, ŷ=0.0052 (WRMSEs=[2.20556893266441], gain=0.000%)
[Stage B] t=78 (I_t=79) | incumbent=1/1
    active=1 | t=78: y=0.7802, ŷ=-0.0373 (WRMSEs=[2.0180518326224837], gain=0.000%)
[Stage B] t=79 (I_t=80) | incumbent=1/1
    active=1 | t=79: y=1.5484, ŷ=-0.0268 (WRMSEs=[2.0784000740660242], gain=0.000%)
[Stage B] t=80 (I_t=81) | incumbent=1/1
    active=1 | t=80: y=0.3812, ŷ=-0.0069 (WRMSEs=[1.9887280212149534], gain=0.000%)
[Stage B] t=81 (I_t=82) | incumbent=1/1
    active=1 | t=81: y=0.3797, ŷ=-0.0020 (WRMSEs=[1.5736219414

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


    active=1 | t=87: y=-0.4975, ŷ=0.0187 (WRMSEs=[0.7625031641633613], gain=0.000%)
[Stage B] t=88 (I_t=89) | incumbent=1/1
    active=1 | t=88: y=2.0000, ŷ=0.0128 (WRMSEs=[1.1040049113330617], gain=0.000%)
[Stage B] t=89 (I_t=90) | incumbent=1/1
    active=1 | t=89: y=-1.9608, ŷ=0.0354 (WRMSEs=[1.3810718651621854], gain=0.000%)
[Stage B] t=90 (I_t=91) | incumbent=1/1
    active=1 | t=90: y=-0.7500, ŷ=0.0129 (WRMSEs=[1.322330798266451], gain=0.000%)
[Stage B] t=91 (I_t=92) | incumbent=1/1
    active=1 | t=91: y=0.5038, ŷ=0.0045 (WRMSEs=[1.2511164872582228], gain=0.000%)
[Stage B] t=92 (I_t=93) | incumbent=1/1
    active=1 | t=92: y=-1.3784, ŷ=0.0100 (WRMSEs=[1.3410894944751084], gain=0.000%)
[Stage B] t=93 (I_t=94) | incumbent=1/1
    active=1 | t=93: y=0.1271, ŷ=-0.0051 (WRMSEs=[1.2941864352076653], gain=0.000%)
[Stage B] t=94 (I_t=95) | incumbent=1/1
    active=1 | t=94: y=2.2843, ŷ=-0.0037 (WRMSEs=[1.4252441087020462], gain=0.000%)
[Stage B] t=95 (I_t=96) | incumbent=1/1
    active=

...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


    active=1 | t=387: y=-2.0202, ŷ=0.0748 (WRMSEs=[1.794419478085827], gain=0.000%)
[Stage B] t=388 (I_t=389) | incumbent=1/1
    active=1 | t=388: y=-0.5155, ŷ=0.0693 (WRMSEs=[1.4705854777827705], gain=0.000%)
[Stage B] t=389 (I_t=390) | incumbent=1/1


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


    active=1 | t=389: y=-0.7254, ŷ=0.0678 (WRMSEs=[1.2834508490383965], gain=0.000%)
[Stage B] t=390 (I_t=391) | incumbent=1/1
    active=1 | t=390: y=-0.5219, ŷ=0.0658 (WRMSEs=[1.0197681489547608], gain=0.000%)
[Stage B] t=391 (I_t=392) | incumbent=1/1
    active=1 | t=391: y=-0.5247, ŷ=0.0643 (WRMSEs=[1.003321829785963], gain=0.000%)
[Stage B] t=392 (I_t=393) | incumbent=1/1
    active=1 | t=392: y=0.1055, ŷ=0.0628 (WRMSEs=[0.9574689401227008], gain=0.000%)
[Stage B] t=393 (I_t=394) | incumbent=1/1
    active=1 | t=393: y=-2.2129, ŷ=0.0629 (WRMSEs=[1.1133250591994743], gain=0.000%)
[Stage B] t=394 (I_t=395) | incumbent=1/1
    active=1 | t=394: y=1.1853, ŷ=0.0571 (WRMSEs=[1.1706107376860049], gain=0.000%)
[Stage B] t=395 (I_t=396) | incumbent=1/1
    active=1 | t=395: y=1.2780, ŷ=0.0600 (WRMSEs=[1.222505639663153], gain=0.000%)
[Stage B] t=396 (I_t=397) | incumbent=1/1
    active=1 | t=396: y=-0.5258, ŷ=0.0630 (WRMSEs=[1.198877493389125], gain=0.000%)
[Stage B] t=397 (I_t=398) | incu

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
